In [1]:
import numpy as np
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config = config)
import keras
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode = 'fine')

class_names = []
with open('cifar100_class_names') as file:
    for line in file:
        string = line.replace(' ', '')
        string = line.replace('\n', '')
        string_list = string.split(',')
        class_names = class_names + string_list
print('There are totally', len(class_names), 'classes')
print('Class names:', class_names)

# preprocessing
X_train = X_train / 255. # .reshape((X_train.shape[0], -1))
X_test = X_test / 255. # .reshape((X_test.shape[0], -1))

def reshape_func(data):
    return data.reshape((data.shape[0], -1))

# model
def create_model(hidden_layers = [128, 64, 32], 
                 activations = ['relu', 'relu', 'relu', 'softmax'], 
                 weight_initializations = ['he_normal', 'he_normal', 'he_normal', 'he_normal'], 
                 learning_rate = 1e-5,
                 loss = 'categorical_crossentropy',
                 optimizer = 'adam', 
                 metrics = ['accuracy']):
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_shape = (3072,), activation = activations[0], kernel_initializer = weight_initializations[0]))
    for i in range(1, len(hidden_layers) - 1):
        model.add(Dense(hidden_layers[i], activation = activations[i], kernel_initializer = weight_initializations[i]))
    model.add(Dense(100, activation = activations[-1]))
    model.compile(loss = loss, optimizer = optimizer, metrics = metrics)
    return model

There are totally 100 classes
Class names: ['beaver', ' dolphin', ' otter', ' seal', ' whale', 'aquarium fish', ' flatfish', ' ray', ' shark', ' trout', 'orchids', ' poppies', ' roses', ' sunflowers', ' tulips', 'bottles', ' bowls', ' cans', ' cups', ' plates', 'apples', ' mushrooms', ' oranges', ' pears', ' sweet peppers', 'clock', ' computer keyboard', ' lamp', ' telephone', ' television', 'bed', ' chair', ' couch', ' table', ' wardrobe', 'bee', ' beetle', ' butterfly', ' caterpillar', ' cockroach', 'bear', ' leopard', ' lion', ' tiger', ' wolf', 'bridge', ' castle', ' house', ' road', ' skyscraper', 'cloud', ' forest', ' mountain', ' plain', ' sea', 'camel', ' cattle', ' chimpanzee', ' elephant', ' kangaroo', 'fox', ' porcupine', ' possum', ' raccoon', ' skunk', 'crab', ' lobster', ' snail', ' spider', ' worm', 'baby', ' boy', ' girl', ' man', ' woman', 'crocodile', ' dinosaur', ' lizard', ' snake', ' turtle', 'hamster', ' mouse', ' rabbit', ' shrew', ' squirrel', 'maple', ' oak', '

In [3]:
class DataClassGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size = 32, n_classes = 100):
        self.X, self.y = X, keras.utils.to_categorical(y, num_classes = n_classes)
        self.batch_size = batch_size
        self.n_classes = n_classes
        
    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        X_batch = self.X[idx * self.batch_size : (idx + 1) * self.batch_size]
        y_batch = self.y[idx * self.batch_size : (idx + 1) * self.batch_size]
        
        return X_batch, y_batch

In [4]:
class_gen = DataClassGenerator(reshape_func(X_train), y_train, len(class_names))
model = create_model()
model.fit_generator(generator = class_gen, epochs = 1)
print('Loss\tAccuracy')
model.evaluate(reshape_func(X_test), keras.utils.to_categorical(y_test))

Epoch 1/1
500/500 [==============================] - 2s 5ms/step - loss: 4.2272 - acc: 0.0565
Loss	Accuracy
10000/10000 [==============================] - 0s 47us/step


[4.0137492988586425, 0.0817]

In [5]:
def func_generator(X, y, batch_size = 32, n_classes = 100):
    y = keras.utils.to_categorical(y_train, num_classes = n_classes)
    
    while 1:
        for idx in range(int(np.ceil(len(X_train) / float(batch_size)))):
            X_batch = X[idx * batch_size : (idx + 1) * batch_size]
            y_batch = y[idx * batch_size : (idx + 1) * batch_size]
            yield X_batch, y_batch

In [6]:
batch_size = 32
model = create_model()
model.fit_generator(generator = func_generator(reshape_func(X_train), y_train, batch_size = batch_size), 
                    steps_per_epoch = int(np.ceil(len(X_train) / float(batch_size))), 
                    epochs = 1)
print('Loss\tAccuracy')
model.evaluate(reshape_func(X_test), keras.utils.to_categorical(y_test))

Epoch 1/1
1563/1563 [==============================] - 6s 4ms/step - loss: 4.1113 - acc: 0.0715
Loss	Accuracy
10000/10000 [==============================] - 0s 47us/step


[3.849954206085205, 0.1099]

In [7]:
model = create_model()
'''
Image data format refers to the representation of batches of images. TensorFlow supports NHWC 
(TensorFlow default) and NCHW (cuDNN default). N refers to the number of images in a batch, 
H refers to the number of pixels in the vertical dimension, W refers to the number of pixels 
in the horizontal dimension, and C refers to the channels (e.g. 1 for black and white, 3 for 
RGB, etc.) Although cuDNN can operate on both formats, it is faster to operate in its default 
format.

The best practice is to build models that work with both NCHW and NHWC as it is common to train
using NCHW on GPU, and then do inference with NHWC on CPU.

The very brief history of these two formats is that TensorFlow started by using NHWC because it 
was a little faster on CPUs. Then the TensorFlow team discovered that NCHW performs better when 
using the NVIDIA cuDNN library. The current recommendation is that users support both formats in 
their models. In the long term, we plan to rewrite graphs to make switching between the formats 
transparent.

ref: https://www.tensorflow.org/performance/performance_guide#use_nchw_imag
'''
def create_cnn():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape = (32, 32, 3), kernel_initializer = 'normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (5, 5), kernel_initializer = 'normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model
model = create_cnn()


kerasGenerator = ImageDataGenerator(featurewise_center = False, 
                               samplewise_center = False, 
                               featurewise_std_normalization = False, 
                               samplewise_std_normalization = False, 
                               zca_whitening = False, 
                               zca_epsilon = 1e-06,
                               rotation_range = 0.0, 
                               width_shift_range = 0.0, # Range for random horizontal shifts
                               height_shift_range = 0.0, # Range for random vertical shifts
                               brightness_range = None, 
                               shear_range = 0.2, # Shear Intensity (Shear angle in counter-clockwise direction in degrees)
                               zoom_range = 0.1, # Range for random zoom
                               channel_shift_range = 0.0, 
                               fill_mode = 'nearest', 
                               cval = 0.0, 
                               horizontal_flip = True, 
                               vertical_flip = False, 
                               rescale = None, 
                               preprocessing_function = None, # Function that will be implied on each input
                               data_format = None, # One of {'channel_first', 'channel_last'}
                               validation_split = 0.0)
kerasGenerator.fit(X_train, augment = True, rounds = 1)
model.fit_generator(generator = kerasGenerator.flow(X_train, keras.utils.to_categorical(y_train), batch_size = 32), 
                    epochs = 1)
print('Loss\tAccuracy')
model.evaluate(X_test, keras.utils.to_categorical(y_test))

Epoch 1/1
1563/1563 [==============================] - 21s 13ms/step - loss: 4.0863 - acc: 0.0796
Loss	Accuracy
10000/10000 [==============================] - 1s 85us/step


[3.5983389251708986, 0.1558]